#### Scrape employee reviews from Indeed 

In [517]:
import urllib
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import re
from time import sleep
import urllib.request
import numpy as np

##### Read companies from fortune.csv

In [518]:
fortune = pd.read_csv('fortune_cos.csv', nrows = 100)
company_list = fortune['title'].values.tolist()

#### Get count of US reviews for the companies

In [525]:
link = "https://www.indeed.com/cmp/{}/reviews?start={}"
results = pd.DataFrame(columns=[])
for company in company_list:
    url = link.format(company, 0)
    html = requests.get(url)
    soup = BeautifulSoup(html.content, 'html.parser', from_encoding="utf-8")
    try:
        result = soup.find('div', class_= "turnstileInfo").find('span', class_ = "cmp-filter-result").b.text
    except:
        result = None
    results = results.append({'company' : company, 'reviews' : result}, ignore_index = True)

In [541]:
results['reviews'] = results['reviews'].apply(lambda x: str(x).replace(",", ""))
results = results.loc[results['reviews'] != 'None', :]
results['reviews'] = pd.to_numeric(results['reviews'])
results = results.loc[results['reviews'] <1000, :].reset_index(drop=True)

##### Number of Companies filtered for sample, keep review count less than 1000

In [542]:
results.shape

(19, 2)

#### Get float rating values from company review main page

In [543]:
link = "https://www.indeed.com/cmp/{}/reviews"
df_float = pd.DataFrame(columns=[])

for company in results.company.tolist():
        url = link.format(company)
        html = requests.get(url)
        soup = BeautifulSoup(html.content, 'html.parser', from_encoding="utf-8")
        
        for each in soup.find_all(class_= "turnstileInfo"):
            try: 
                overall_rating = each.find('span', {'class':"cmp-average-rating" }).text.replace('\n', '')
            except:
                overall_rating = None
            try: 
                review_count = each.find('div', {'class':"cmp-sidebar-section cmp-bordered-box" }).\
                find('div', {'class':"cmp-note" }).text
            except:
                review_count = None
            try: 
                overall_rating = each.find('span', {'class':"cmp-average-rating" }).text.replace('\n', '')
            except:
                overall_rating = None
            try:
                worklife_rating = soup.find('div', {'class':"cmp-sidebar-section cmp-bordered-box" }).\
                find_all('span', {'class':"cmp-ReviewCategories-rating"})[0].text
            except:
                worklife_rating = None
            try:
                compensation_rating = soup.find('div', {'class':"cmp-sidebar-section cmp-bordered-box" }).\
                find_all('span', {'class':"cmp-ReviewCategories-rating"})[1].text
            except:
                compensation_rating = None
            try:
                jobsecurity_rating = soup.find('div', {'class':"cmp-sidebar-section cmp-bordered-box" }).\
                find_all('span', {'class':"cmp-ReviewCategories-rating"})[2].text
            except:
                jobsecurity_rating = None
            try:
                mgmt_rating = soup.find('div', {'class':"cmp-sidebar-section cmp-bordered-box" }).\
                find_all('span', {'class':"cmp-ReviewCategories-rating"})[3].text
            except:
                mgmt_rating = None
            try:
                culture_rating = soup.find('div', {'class':"cmp-sidebar-section cmp-bordered-box" }).\
                find_all('span', {'class':"cmp-ReviewCategories-rating"})[4].text
            except:
                culture_rating = None
                
            
            df_float = df_float.append({'company' : company, 'overall' : overall_rating, "globalreview_count" : review_count,\
                           'worklife_rating' : worklife_rating, "compensation_rating" : compensation_rating,\
                           'jobsecurity_rating' : jobsecurity_rating, "mgmt_rating" : mgmt_rating,\
                           'culture_rating' : culture_rating}, ignore_index = True)
           
        sleep(1)

In [544]:
df_float.head(10)

,company,compensation_rating,culture_rating,globalreview_count,jobsecurity_rating,mgmt_rating,overall,worklife_rating
0,Berkshire Hathaway,3.8,4.0,Based on 104 reviews,3.6,3.9,4.1,4.0
1,Walgreens Boots Alliance,3.5,3.2,Based on 279 reviews,3.1,3.1,3.4,3.2
2,Fannie Mae,4.1,3.8,Based on 920 reviews,3.4,3.5,4.0,4.0
3,Alphabet,3.8,3.3,None,3.8,3.0,None,4.5
4,Phillips 66,None,None,None,None,None,3.8,None
5,Freddie Mac,3.8,3.7,Based on 586 reviews,3.4,3.4,3.9,4.0
6,Marathon Petroleum,3.6,3.5,Based on 590 reviews,3.2,3.4,3.8,3.7
7,DowDuPont,None,None,None,None,None,4.3,None
8,Archer Daniels Midland,3.9,3.5,Based on 714 reviews,3.4,3.3,3.8,3.3
9,United Technologies,3.7,3.4,Based on 874 reviews,3.3,3.3,3.8,3.6


##### Number of reviews for each company

In [545]:
n_rows = 0
print('Company name : Number of reviews \n')
for company in results.company.tolist():
    page_count = (results.loc[results['company'] == company, 'reviews'].values[0])
    print(company,":", page_count)
    n_rows = n_rows + page_count
print('\n', 'Total reviews for all companies considered:', n_rows)

Company name : Number of reviews 

Berkshire Hathaway : 101
Walgreens Boots Alliance : 252
Fannie Mae : 915
Alphabet : 1
Phillips 66 : 385
Freddie Mac : 581
Marathon Petroleum : 575
DowDuPont : 3
Archer Daniels Midland : 510
United Technologies : 753
Goldman Sachs Group : 781
Facebook : 272
Tech Data : 389
TIAA : 681
World Fuel Services : 79
Exelon : 480
Massachusetts Mutual Life Insurance : 813
ConocoPhillips : 677
CHS : 180

 Total reviews for all companies considered: 8428


##### Get all individual links to be scraped by BeautifulSoup. Each page has 20 reviews.

In [546]:
link = "https://www.indeed.com/cmp/{}/reviews?start={}"
df_text = pd.DataFrame(columns=[])

#results = int(soup.find('div', class_= "turnstileInfo").find('span', class_ = "cmp-filter-result").b.text)

for company in results.company.tolist():
    page_count = results.loc[results['company'] == company, 'reviews'].values[0]
    for step in range(0,page_count-19,20):
        url = link.format(company, step)
        print(url)

https://www.indeed.com/cmp/Berkshire Hathaway/reviews?start=0
https://www.indeed.com/cmp/Berkshire Hathaway/reviews?start=20
https://www.indeed.com/cmp/Berkshire Hathaway/reviews?start=40
https://www.indeed.com/cmp/Berkshire Hathaway/reviews?start=60
https://www.indeed.com/cmp/Berkshire Hathaway/reviews?start=80
https://www.indeed.com/cmp/Walgreens Boots Alliance/reviews?start=0
https://www.indeed.com/cmp/Walgreens Boots Alliance/reviews?start=20
https://www.indeed.com/cmp/Walgreens Boots Alliance/reviews?start=40
https://www.indeed.com/cmp/Walgreens Boots Alliance/reviews?start=60
https://www.indeed.com/cmp/Walgreens Boots Alliance/reviews?start=80
https://www.indeed.com/cmp/Walgreens Boots Alliance/reviews?start=100
https://www.indeed.com/cmp/Walgreens Boots Alliance/reviews?start=120
https://www.indeed.com/cmp/Walgreens Boots Alliance/reviews?start=140
https://www.indeed.com/cmp/Walgreens Boots Alliance/reviews?start=160
https://www.indeed.com/cmp/Walgreens Boots Alliance/reviews?st

#### Get all text reviews for companies considered

In [547]:
link = "https://www.indeed.com/cmp/{}/reviews?start={}"
df_text = pd.DataFrame(columns=[])

#results = int(soup.find('div', class_= "turnstileInfo").find('span', class_ = "cmp-filter-result").b.text)

for company in results.company.tolist():
    page_count = results.loc[results['company'] == company, 'reviews'].values[0]
    
    for step in range(0,page_count-19,20):
        url = link.format(company, step)
        html = requests.get(url)
        soup = BeautifulSoup(html.content, 'html.parser', from_encoding="utf-8")

        
        for each in soup.find(class_= "cmp-inline-block cmp-review-content"):
            try: 
                title = each.find_all('div', {'class':"cmp-review-title" })[0].text
            except:
                title = None
            try:
                author = each.find_all(class_="cmp-review-title" )[0].\
                find('meta').get('content')
            except:
                author = None
            try:
                employee_type = each.find_all('span', class_ = "cmp-reviewer-job-title")[0].text
            except:
                employee_type = None
            try:
                text = each.find_all('span', {'class' :"cmp-review-text"})[0].text
            except:
                text = None
            try:
                pros = each.find_all('div', {'class' :"cmp-review-pro-text"})[0].text
            except:
                pros = None
            try:
                cons = each.find_all('div', {'class' :"cmp-review-con-text"})[0].text
            except:
                cons = None
            try:
                location = each.find_all('span', class_ = "cmp-reviewer-job-location")[0].text
            except:
                location = None
            try:
                date = each.find_all('span', class_ = "cmp-review-date-created")[0].text
            except:
                date = None
                
            df_text= df_text.append({'company' : company,'title' : title, 'author' : author, \
                                     'employee_type' : employee_type, 'text': text, 'pros' : pros, \
                                     'cons' : cons, 'location' : location,'date' : date}, ignore_index = True)
            
        sleep(1)

##### Text review dataframe

In [548]:
df_text.head(20)

,author,company,cons,date,employee_type,location,pros,text,title
0,"Sales and Marketing, Sales Support Associate",Berkshire Hathaway,no room for growth,"April 23, 2013","Sales and Marketing, Sales Support Associate (...","New York, NY","helpful, energetic and friendly working enviro...",Learned to grow into a leadership role and men...,Motivating Workplace
1,None,Berkshire Hathaway,None,None,None,None,None,None,None
2,Realtor,Berkshire Hathaway,None,"September 26, 2018",Realtor (Current Employee) –,"Bonita Springs, FL",None,Berkshire is the top real estate company in th...,Top Company
3,Assistant,Berkshire Hathaway,None,"September 3, 2018",Assistant (Former Employee) –,"Moreland Hills, OH",Great hours. Good people.,Working at Berkshire Hathaway taught me what i...,What I learned
4,UPS Driver Helper,Berkshire Hathaway,Takes a toll on Your body,"August 22, 2018",UPS Driver Helper (Former Employee) –,"Omaha, NE",Its a workout,Sometimes when i worked there i felt like i wa...,Can be fun and competitive
5,SALES COORDINATOR SHAW INDUSTRIES A DIVISION,Berkshire Hathaway,no advancement,"August 15, 2018",SALES COORDINATOR SHAW INDUSTRIES A DIVISION (...,"Monument, CO",Pay/benefits are good,People are friendly...hard to advance though. ...,Daily Job
6,Senior Industrial Hygiene and Safety Leader,Berkshire Hathaway,"Health care, no long term disability","August 14, 2018",Senior Industrial Hygiene and Safety Leader (C...,"Cleveland, TN",We are all in this together and a family type ...,I like all the people I work with and for. Not...,I've worked for my company for 24 years and th...
7,Data Entry,Berkshire Hathaway,Certified and relaxed healthcare,"July 13, 2018",Data Entry (Current Employee) –,"Omaha, NE",Free lunches,The Hiring manager to the BerkshirehathawayINC...,Cool and well relaxed
8,REALTOR,Berkshire Hathaway,None,"July 2, 2018",REALTOR (Former Employee) –,"York, PA",None,Working with berkshire hathaway was in a way g...,It is what you make it!
9,assistant in residential real estate,Berkshire Hathaway,The hours could be extremely long and the pres...,"June 26, 2018",assistant in residential real estate (Former E...,"Newport Beach, CA","Worked independently, and set my own schedule....",Originally worked as an real estate agent in p...,Opportunity to work independently but with in ...


#### Merge Float and text dataframe

In [549]:
df = pd.merge(df_text, df_float, on = 'company', how = 'left')

In [550]:
print("Shape of final dataframe : {}".format(df.shape))

Shape of final dataframe : (10019, 16)


In [551]:
df.to_csv('reviews.csv', header = True)

In [552]:
df['company'].value_counts()

Fannie Mae                             1092
Massachusetts Mutual Life Insurance     968
Goldman Sachs Group                     940
United Technologies                     895
TIAA                                    825
ConocoPhillips                          799
Freddie Mac                             699
Marathon Petroleum                      678
Archer Daniels Midland                  601
Exelon                                  582
Tech Data                               460
Phillips 66                             459
Facebook                                317
Walgreens Boots Alliance                291
CHS                                     219
Berkshire Hathaway                      122
World Fuel Services                      72
Name: company, dtype: int64

### Below cells for Rough Work

In [218]:
soup.find(class_= "cmp-inline-block cmp-review-content").find_all('div', {'class' :"cmp-review-pro-text"})[2].text

'Free food and snacks, very open and diverse environment, great company'

In [223]:
soup.find(class_= "cmp-inline-block cmp-review-content").find_all('div', {'class' :"cmp-review-con-text"})[1].text

'Everything'

In [ ]:
cmp-review-container  div
cmp-review   div
cmp-review-content-container  div
cmp-review-description  div
cmp-review-text  span

In [243]:
soup.find(class_= "cmp-inline-block cmp-review-content").find_all('span', {'class':"cmp-reviewer-job-title" })[1]\
.text.replace(u'\xa0', u' ')

'Spam Analyst (Current Employee) –  '

In [295]:
soup.find(class_= "cmp-inline-block cmp-review-content").find_all('span', class_ = "cmp-filter-result")

[]

In [ ]:
turnstileInfo
cmp-inline-block cmp-review-content
clearfix
cmp-review-topic-filter cmp-review-topic-filter-buttons
cmp-filter-result

In [314]:
int(soup.find('div', class_= "turnstileInfo").find('span', class_ = "cmp-filter-result").b.text)
#find_all('div', class_= "cmp-filter-result")

272

In [306]:
soup.find(class_= "turnstileInfo")

<div class="turnstileInfo" data-tn-application="acme-webapp" data-tn-olth="ad40ce22551837d40d3253ae0aa5211b" data-tn-originlogid="1cotqfmu15ne2800" data-tn-originlogtype="acmePage" data-tn-page="reviews" dir="ltr" id="cmp-root" itemscope="" itemtype="http://schema.org/LocalBusiness"><div id="cmp-autocomplete-header"><div class="cmp-jumbotron cmp-autocomplete-curated" data-tn-section="company-search-header" id="cmp-new-search-header"><div class="cmp-search-header-side-padding" id="cmp-autocomplete-search-card-header"><div class="cmp-card"><form action="/cmp" id="cmp-autocomplete-cs-form-header" method="get"><div id="search-by-company-elem-header"><input name="from" type="hidden" value="reviews-cmp-search-header"/><input autocomplete="off" class="cmp-autocomplete-input-header" data-initial="" data-search="" id="search-by-company-input-header" maxlength="512" name="q" placeholder="Company name" type="text" value=""/><button data-tn-element="find-companies-button" data-tn-link="" type="sub

In [193]:
soup.find_all('div', {'class' :"cmp-inline-block cmp-review-content"}).find('div', {'class':"cmp-review-title" })

AttributeError: ResultSet object has no attribute 'find'. You're probably treating a list of items like a single item. Did you call find_all() when you meant to call find()?

In [198]:
soup.find(class_= "cmp-inline-block cmp-review-content").find_all(class_="cmp-review-title" )[0]

<div class="cmp-review-title"><span>Overall great company to work for!</span><span itemprop="author" itemscope="" itemtype="http://schema.org/Person"><meta content="Administrative Assistant" itemprop="name"/></span></div>

In [ ]:
find('div', {'class':"cmp-review-title" })[0].find('meta').get('content')

In [188]:
soup.find(class_= "cmp-inline-block cmp-review-content").find('div', {'class':"cmp-review-title" }).find('meta').get('content')

'Administrative Assistant'

In [ ]:
'span', {'itemprop':"meta content"}

In [158]:
soup.find(class_= "cmp-inline-block cmp-review-content").find_all('div', {'class':"cmp-review-title" })[0]

<div class="cmp-review-title"><span>Overall great company to work for!</span><span itemprop="author" itemscope="" itemtype="http://schema.org/Person"><meta content="Administrative Assistant" itemprop="name"/></span></div>

In [97]:
soup.find('div', {'class':"cmp-inline-block cmp-review-content" }).find_all('div', {'class':"cmp-review-title" })[0].text

'Good'

In [ ]:
https://www.indeed.com/cmp/Salesforce/reviews?start=20

In [ ]:
review title
reviewer designation
review date
review text
review pros
review cons

In [ ]:

turnstileInfo
cmp-inline-block cmp-review-content
clearfix
cmp-review-topic-filter cmp-review-topic-filter-buttons
cmp-filter-result

In [117]:
soup.find(class_= "turnstileInfo").prettify()

#find('div', {'class':"cmp-inline-block cmp-review-content"})
#find('div', {'class':"clearfix"})

#find('div', {'class':"cmp-review-topic-filter cmp-review-topic-filter-buttons"}).\
#find('span', {'class':"cmp-filter-result"})

'<div class="turnstileInfo" data-tn-application="acme-webapp" data-tn-olth="ad40ce22551837d40d3253ae0aa5211b" data-tn-originlogid="1corodof45j5s800" data-tn-originlogtype="acmePage" data-tn-page="reviews" dir="ltr" id="cmp-root" itemscope="" itemtype="http://schema.org/LocalBusiness">\n <div id="cmp-autocomplete-header">\n  <div class="cmp-jumbotron cmp-autocomplete-curated" data-tn-section="company-search-header" id="cmp-new-search-header">\n   <div class="cmp-search-header-side-padding" id="cmp-autocomplete-search-card-header">\n    <div class="cmp-card">\n     <form action="/cmp" id="cmp-autocomplete-cs-form-header" method="get">\n      <div id="search-by-company-elem-header">\n       <input name="from" type="hidden" value="reviews-cmp-search-header"/>\n       <input autocomplete="off" class="cmp-autocomplete-input-header" data-initial="" data-search="" id="search-by-company-input-header" maxlength="512" name="q" placeholder="Company name" type="text" value=""/>\n       <button data